In [6]:
import os
import datetime
import firebirdsql
import pandas as pd
from queue import Queue
import concurrent.futures
from openpyxl import Workbook
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

In [7]:
# ===================== Funções de Conexão =====================
def get_firebird_connection():
    # Ajuste os parâmetros conforme sua configuração, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [3]:
conn = get_firebird_connection()
cursor = conn.cursor()

query = """
            SELECT NOME, OBS
            FROM CLIENTE
            ORDER BY CDCLIENTE
        """
        
cursor.execute(query)
rows = cursor.fetchall()

In [4]:
import re

def sanitize_string(value):
    """
    Remove caracteres ilegais para XML (usado pelo OpenPyXL) de uma string.
    """
    # Essa expressão regular remove caracteres com códigos 0-8, 11, 12 e 14-31.
    illegal_xml_chars_re = re.compile(r'[\000-\010\013\014\016-\037]')
    return illegal_xml_chars_re.sub("", value)

In [5]:
wb = Workbook()
ws = wb.active

for cliente in rows:
    processed_row = []
    for column in cliente:
        # Se for do tipo bytes, decodifica-o para string
        if isinstance(column, bytes):
            try:
                decoded = column.decode("iso-8859-1", errors="replace")
            except UnicodeDecodeError:
                decoded = column.decode("utf-8", errors="replace")
            processed_row.append(sanitize_string(decoded))
        else:
            processed_row.append(column)
    # Adiciona a linha completa ao worksheet
    ws.append(processed_row)

# Salva o arquivo Excel
output_filename = "clientes-obs.xlsx"
wb.save(output_filename)
print(f"Arquivo Excel salvo como '{output_filename}'")

Arquivo Excel salvo como 'clientes-obs.xlsx'


In [ ]:
# ===================== Criação do Excel =====================
wb = Workbook()
ws = wb.active

# Definindo o header na ordem especificada
header = [
    "CDCLIENTE",
    "NOME",
    "DESCONTOHBL",
    "ENDERECO",
    "BAIRRO",
    "CIDADE",
    "ESTADO",
    "CEP",
    "VALORAFATURAR",
    "VALORFATURADO",
    "DTCADASTRO",
    "DTULTIMOMOV",
    "TEMENDENVIO",
    "TEMENDNF",
    "PRAZOPGTO",
    "CDTIPO",
    "CDDESCONTO",
    "TIPO",
    "CDLOCAL",
    "CDLOCALCOBRANCA",
    "PRAZOPEDIDO",
    "LIMITECREDITO",
    "SITUACAO",
    "CPF_CNPJ",
    "NOMEFANTASIA",
    "DTFUNDACAO",
    "EMAIL",
    "INSCRICAOMUNICIPAL",
    "RESPONSAVELPGTO",
    "FONERESPPGTO",
    "RESPONSAVELCOMPRA",
    "FONERESPCOMPRA",
    "NUMREGJUNTACOMERCIAL",
    "FILIACAPAI",
    "FILIACAOMAE",
    "DTNASCIMENTO",
    "IE",
    "UF_RG",
    "OBS",
    "INFESCOLHIDA",
    "CDATIVIDADECOMERCIAL",
    "CDAGENCIAPUBL",
    "PORCENTAGEM_AGENCIA",
    "TEMP_VALDUPLICATAS",
    "TEMP_VALDEBITO",
    "DIASPRAZOPAG",
    "CDTIPONFSAIDA",
    "CDFUNC",
    "CDMUNIC",
    "ENDER_NUMERO",
    "PONTORF",
    "APELIDO",
    "PAF_CODIGO_CLIENTE_COMPOSTO",
    "PAF_CODIGO_EMPRESA",
    "PAF_MODO_BD",
    "PAF_CPF",
    "PAF_SALDO_DISPONIVEL",
    "RELACIONAMENTOATIVO",
    "RELACIONAMENTOVALIDADE",
    "CDCLIENTEINDICACAO",
    "CADASTROCOMPLETO",
    "HABILITAFORMAPAG",
    "TEMPOATUACAORAMO",
    "CONSUMIDORFINAL",
    "DTULTICONTATO"
]

# Insere o header como primeira linha do Excel
ws.append(header)

# Insere as linhas de dados (cada linha já com 38 colunas)
# Para cada registro (cliente), monta uma linha processada
for cliente in rows:
    processed_row = []
    for column in cliente:
        # Se for do tipo bytes, decodifica-o para string
        if isinstance(column, bytes):
            try:
                decoded = column.decode("iso-8859-1", errors="replace")
            except UnicodeDecodeError:
                decoded = column.decode("utf-8", errors="replace")
            processed_row.append(sanitize_string(decoded))
        else:
            processed_row.append(column)
    # Adiciona a linha completa ao worksheet
    ws.append(processed_row)

# Salva o arquivo Excel
output_filename = "clientes.xlsx"
wb.save(output_filename)
print(f"Arquivo Excel salvo como '{output_filename}'")

In [8]:
# Definindo o período: de 01/03/2024 a 28/02/2025
start_date = datetime.date(2024, 3, 1)
end_date = datetime.date(2025, 2, 28)

# Conectando e executando a query para extrair os dados da tabela APAGAR
conn = get_firebird_connection()
query = """
    SELECT DTVENCIMENTO, VALOR
    FROM APAGAR
    WHERE DTVENCIMENTO BETWEEN ? AND ?
"""
# Lê os dados da query para um DataFrame
df = pd.read_sql(query, conn, params=(start_date, end_date))
conn.close()

# Converte a coluna DTVENCIMENTO para datetime (assumindo que as datas estejam no formato dd/mm/yy)
df['DTVENCIMENTO'] = pd.to_datetime(df['DTVENCIMENTO'], dayfirst=True)

# Cria colunas auxiliares para o agrupamento:
# 'year_month' conterá o período do mês (ex.: 2024-03)
# 'day' conterá o dia do mês (1,2,...,31)
df['year_month'] = df['DTVENCIMENTO'].dt.to_period('M')
df['day'] = df['DTVENCIMENTO'].dt.day

# Agrupa por 'year_month' e 'day', calculando a média do VALOR para cada dia
grouped = df.groupby(['year_month', 'day'])['VALOR'].mean().reset_index()
grouped = grouped.rename(columns={'VALOR': 'avg_valor'})

# Para cada mês, seleciona o dia com a média de gasto mais baixa
result = grouped.loc[grouped.groupby('year_month')['avg_valor'].idxmin()].reset_index(drop=True)

# Exibe o resultado
print("Dia de menor gasto (média) por mês:")
print(result)

C:\Users\Mateus\AppData\Local\Temp\ipykernel_22700\3200533431.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(start_date, end_date))


Dia de menor gasto (média) por mês:
   year_month  day   avg_valor
0     2024-03    2   56.500000
1     2024-04    6   20.000000
2     2024-05   12  243.972857
3     2024-06   22  297.027500
4     2024-07   21  131.333333
5     2024-08   21   19.750000
6     2024-09   21   65.003333
7     2024-10   12  173.192000
8     2024-11   16   75.000000
9     2024-12   21  153.735417
10    2025-01   25  228.402000
11    2025-02   23  143.665000
